In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
# from rsna_retro.train3d import *
from rsna_retro.train_adjacent import *

Loading imports


In [2]:
torch.cuda.set_device(4)

## Train

In [3]:
name = 'train_adjacent_5slice'

In [4]:
dbunch = get_adj_data(512, 128, Meta.splits_stg1, num_adj=2)
learn = get_learner(dbunch, partial(xresnet18, c_in=5))

In [5]:
do_fit(learn, 12, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.134521,0.186452,0.954977,0.900567,22:08
1,0.118813,0.200285,0.951811,0.882500,22:16
2,0.111451,0.116762,0.965285,0.927697,22:15
3,0.104367,0.122914,0.965084,0.926488,22:08
4,0.099635,0.110994,0.967174,0.932676,22:13
5,0.094237,0.103127,0.969122,0.935476,22:05
6,0.092475,0.100161,0.970053,0.937934,22:13
7,0.086409,0.096351,0.970546,0.938990,22:17
8,0.080964,0.095626,0.970872,0.940747,22:10
9,0.074496,0.086817,0.973605,0.946629,21:34


In [6]:
learn.dbunch = get_adj_data(512, 256, Meta.splits_stg1, num_adj=2)
do_fit(learn, 12, 4e-3)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.080543,0.092562,0.971706,0.942772,21:51
1,0.076271,0.086233,0.973523,0.947011,21:56
2,0.074624,0.085056,0.973141,0.945929,21:51
3,0.073481,0.082197,0.974478,0.949443,22:03
4,0.069635,0.083229,0.974824,0.950029,22:04
5,0.068641,0.080364,0.975008,0.950258,21:43
6,0.065503,0.077854,0.975721,0.952282,21:39
7,0.063969,0.078598,0.975712,0.952588,22:14
8,0.060533,0.080197,0.976292,0.953110,21:51
9,0.054940,0.079564,0.976551,0.953670,22:04


In [7]:
learn.dbunch = get_adj_data(256, 384, Meta.splits_stg1, num_adj=2, img_dir=path_jpg)
do_fit(learn, 4, 4e-4)
learn.save(f'runs/{name}-3')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.065313,0.076063,0.976158,0.953492,51:20
1,0.057907,0.078051,0.976754,0.953593,51:40
2,0.053618,0.074203,0.977340,0.955236,52:47
3,0.054038,0.074516,0.977312,0.954994,53:27


## Submission

In [5]:
learn.load(f'runs/{name}-3')
sub_fn = f'subm/{name}'

In [6]:
# learn.dbunch = get_data_gen(Meta.df_tst, bs=512, sz=None, with_aug=False, tst_dir='tst_jpg256')
learn.dbunch = get_adj_test_data(256, sz=384, tst_dir='tst_jpg', num_adj=2)

In [7]:
preds,targs = learn.get_preds()

In [8]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [9]:
api.competition_submit(f'{sub_fn}.csv', f'{name} adjacent 5c', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.87MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [10]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14277700,
 'totalBytes': 27277209,
 'date': '2020-01-24T00:32:08.123Z',
 'description': 'train_adjacent_5slice adjacent 5c',
 'errorDescription': None,
 'fileName': 'train_adjacent_5slice.csv',
 'publicScore': '0.55114',
 'privateScore': '0.05921',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14277700/14277700.raw'}